# Analyzing the output of a polychrom simulation

Running example.py generates a few different output files, one with the starting conformation, one with the intial arguments, one with the appliced forces, one with the forcekit, and finally the blocks. Here, we ran 10 blocks, each of 100 timesteps each, and set the max data length to 5.

In [2]:
ls trajectory/

applied_forces_0.h5  blocks_5-9.h5                 initArgs_0.h5
blocks_0-4.h5        forcekit_polymer_chains_0.h5  starting_conformation_0.h5


In [1]:
import pandas as pd

In [2]:
#Here are the import statements sameer used in his notebook to analyze a simulation
import os
import importlib as imp
from collections import defaultdict
import h5py
import json
from copy import deepcopy

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import numpy.ma as ma
import pandas as pd
import scipy
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d, interp2d

import polychrom
from polychrom import polymer_analyses, contactmaps, polymerutils
from polychrom.hdf5_format import list_URIs, load_URI

import nglutils as ngu
import nglview as nv

In [2]:
%matplotlib notebook

## Reading data

list_URIs will list all of the files starting with `block` with one URI (universal resource identifier aka path) per conformation

In [6]:
trajs = list_URIs('trajectory')
print(trajs)

['trajectory/blocks_0-4.h5::0', 'trajectory/blocks_0-4.h5::1', 'trajectory/blocks_0-4.h5::2', 'trajectory/blocks_0-4.h5::3', 'trajectory/blocks_0-4.h5::4', 'trajectory/blocks_5-9.h5::5', 'trajectory/blocks_5-9.h5::6', 'trajectory/blocks_5-9.h5::7', 'trajectory/blocks_5-9.h5::8', 'trajectory/blocks_5-9.h5::9']


In [4]:
#TODO: figure out how to extract initial parameters from other h5 files

In [7]:
data = load_URI(trajs[0])

In [8]:
data

{'pos': array([[-0.47, -1.51, -0.37],
        [-0.45, -1.95,  0.55],
        [ 0.45, -2.32,  0.71],
        ...,
        [ 2.05, -1.76,  0.76],
        [ 1.21, -1.56,  0.26],
        [ 0.5 , -1.88, -0.22]], dtype=float32),
 'block': 0,
 'kineticEnergy': 1.8880411331282307,
 'potentialEnergy': 2.5845410077802025,
 'time': 6.356380507349968}

In [9]:
ls trajectory

applied_forces_0.h5  blocks_5-9.h5                 initArgs_0.h5
blocks_0-4.h5        forcekit_polymer_chains_0.h5  starting_conformation_0.h5


In [3]:
with h5py.File("trajectory/initArgs_0.h5", 'r') as f:
    print(f.attrs.keys())

<KeysViewHDF5 ['GPU', 'N', 'PBCbox', 'collision_rate', 'error_tol', 'integrator', 'length_scale', 'mass', 'max_Ek', 'platform', 'precision', 'save_decimals', 'temperature', 'verbose']>
